# World Development Indicators and Life Expectancy in 2010

This notebook examines adult literacy worldwide and what factors lead to having a high life expectancy in the year 2010. To start, we load in the relevant libraries and SQLite database using SQLalchemy and print the table and column names.

In [ ]:
# import relevant libraries

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as sqla

In [ ]:
# Connect to SQLite database found in database.sqlite

metadata = sqla.MetaData()

engine =  sqla.create_engine("sqlite:///../input/database.sqlite")

table_names = engine.table_names()

tables = dict()

for table in table_names:
    print("\n"+table+" columns:\n")
    tables[table] = sqla.Table(table, metadata, autoload=True, autoload_with=engine)
    for column in tables[table].c.keys():
        print(column)

## Initial Exploratory Analysis

Once we have created an engine and loaded the tables from the database, we can then do an exploratory analysis of the data. We want to examine life expectancy, but first we must find which indicator in our table refers to this field. We use the Indicators and Series table to find indicators with the word "life expectancy" in them, and print the name of the indicator, the IndicatorCode, and a description of the Indicator.

In [ ]:
# Create SQL query function (We'll use a similar query again later)

# Select distinct IndicatorNames (to avoid repeating the same indicator for each country)

def find_indicator_strings(indicator):
    stmt = sqla.select([tables["Indicators"].c.IndicatorName.distinct(), 
                        tables["Indicators"].c.IndicatorCode,
                        tables["Series"].c.LongDefinition])

    # Use JOIN to find description under series name

    stmt = stmt.select_from(
        tables["Indicators"].join(tables["Series"], 
                                 tables["Indicators"].c.IndicatorCode == tables["Series"].c.SeriesCode)
    )

    # Find indicators that have indicator somewhere

    return stmt.where(tables["Indicators"].c.IndicatorName.ilike("%"+indicator+"%"))

stmt = find_indicator_strings("life expectancy")

# Connect to the engine and execute the statement.

conn = engine.connect()

for result in conn.execute(stmt):
    print(result.IndicatorName, result.IndicatorCode)
    print(result.LongDefinition)
    print("\n")

conn.close()

From these queries, we have identified that the indicator that most closely matches our query is the life expectancy at birth, which has the code SP.DYN.LE00.IN. We then use this to select the life expectancy for the year 2010 and plot it on a histogram.

In [ ]:
%matplotlib inline
# Create SQL query

stmt = sqla.select([tables["Indicators"].c.CountryName, tables["Indicators"].c.Value.label("LifeExpectancy")])


# Use to avoid selecting regions instead of countries
stmt = stmt.select_from(tables["Indicators"].join(tables["Country"],
                                                 tables["Country"].c.CountryCode == tables["Indicators"].c.CountryCode))

stmt = stmt.where(sqla.and_(tables["Country"].c.Region.isnot(""),
                            tables["Indicators"].c.IndicatorCode == "SP.DYN.LE00.IN",
                            tables["Indicators"].c.Year == 2010)
    )


conn = engine.connect()

# Load into pandas dataframe

life_exp = pd.read_sql_query(stmt, conn)

print(life_exp.info())
print(life_exp.describe())


plt.hist(life_exp["LifeExpectancy"], bins = 14)
plt.xlabel("Life Expectancy from Birth")
plt.ylabel("Count")

conn.close()

From our histogram and description of the dataset, we can see that the mean life expectancy in 2010 was about 70, although the distribution is skewed left since the median is actually almost 73 years.

## Exploring possible predictors

We then would like to query for other available indicators so that later we can use machine learning to try to understand what may predict life expectancy. Using our earlier function we defined to try to find several different predictors. We would like to investigate how public health (in terms of infant mortality and environmental factors) and economics affect life expectancy. Here, we look for infant mortality, fertility, population density, GDP per capita, Inflation, PM2.5 Exposure, and CO2 emissions.

In [ ]:
# Create list to look through
string_list = ["infant", "fertility", "population density", "GDP per capita", "Inflation", "PM2.5", "CO2 emissions"]

conn = engine.connect()

for string in string_list:
    print(string+"\n")
    stmt = find_indicator_strings(string)
    for result in conn.execute(stmt):
        print(result.IndicatorName, result.IndicatorCode)
        print(result.LongDefinition)
        print("\n")

conn.close()
    

Here, we select Mortality rate, infant (per 1,000 live births) SP.DYN.IMRT.IN, Fertility rate, total (births per woman) SP.DYN.TFRT.IN, Population density (people per sq. km of land area) EN.POP.DNST, GDP per capita, PPP (constant 2011 international dollars) NY.GDP.PCAP.PP.KD, PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) EN.ATM.PM25.MC.M3, and CO2 emissions (kt) EN.ATM.CO2E.KT. (In practice, we should likely select many more, but this notebook is meant to be a simple demonstration.)

In [ ]:
%matplotlib inline

# Create dictionary of codes we wish to query to human readable names

code_dict = {"SP.DYN.LE00.IN" : "LifeExpectancy", 
             "SP.DYN.IMRT.IN" : "InfantMort",
             "SP.DYN.TFRT.IN" : "Fertility",
             "EN.POP.DNST" : "PopDens",
             "NY.GDP.PCAP.PP.KD" : "GDPperCap",
             "NY.GDP.DEFL.KD.ZG" : "Inflation",
             "EN.ATM.PM25.MC.M3" : "PM2.5Exp",
             "EN.ATM.CO2E.KT" : "CO2"}

# Create SQL query

stmt = sqla.select([tables["Indicators"].c.CountryName, 
                    tables["Indicators"].c.IndicatorCode.label("Indicator"),
                    tables["Indicators"].c.Value])


# Use to avoid selecting regions instead of countries
stmt = stmt.select_from(tables["Indicators"].join(tables["Country"],
                                                 tables["Country"].c.CountryCode == tables["Indicators"].c.CountryCode))

stmt = stmt.where(sqla.and_(tables["Country"].c.Region.isnot(""),
                            tables["Indicators"].c.IndicatorCode.in_(list(code_dict.keys())),
                            tables["Indicators"].c.Year == 2010)
    )

conn = engine.connect()

life_exp = pd.read_sql_query(stmt, conn)

# Change codes to readable names
life_exp["Indicator"].replace(code_dict, inplace=True)

# Change from long to wide format
life_exp = life_exp.pivot(index="CountryName", columns = "Indicator", values = "Value")

# Remove NA values
life_exp.dropna(inplace=True)

print(life_exp.info())
print(life_exp.describe())



conn.close()

Although we lost some data by removing missing values, we now have a dataset that we can explore further to see how they might predict Life Expectancy. We first may want to check which predictors are correlated with LifeExpetancy and which predictors are correlated with each other. For the latter, we want to check for possible covariance. For example, countries that are richer might emit more CO2, so GDP per capita might also be related to CO2 emissions. 

In [ ]:
%matplotlib inline

sns.heatmap(life_exp.corr(), square=True, cmap='RdYlBu')

Certainly, each variable is highly correlated with itself. However, it seems CO2 emissions, PM2.5 Exposure, and population density are only weakly correlated to each other variable. Of the variables shown here, Fertility, GDP per Capita, Inflation, and Infant Mortality seem most strongly correlated with LifeExpectancy. However, each of these also seem to be highly interrelated as shown by the more intense blue/orange colors. 

## Machine learning with regression

To test what might predict LifeExpectancy more rigorously, we apply machine learning using regression in scikit-learn. Here, Ordinary Least Squares linear regression is likely going to be inadequate since several terms in our predictors seem to be correlated to each other. To avoid this problem, we use elastic net regularized regression. Elastic net uses a hyperparameter l1 ratio that controls how much the square and the absolute value of the coefficient are weighted and a hyperparameter alpha that is the penalty applied to coefficients.

Before using regression, we need to first split the data into the independent and dependent variables. Then, we split the data into testing and training data.Based on our summary of the data above, most of the variables are on different scales, so we normalize the data by their means and standard deviations. We do 5-fold cross validation to minimize overfitting and do a search of which hyperparameter will best fit the data. 

We check the fit through the $R^2$ value on the test set and the best values of the hyperparameters.

In [ ]:
from sklearn import model_selection
from sklearn import linear_model

# Split data into dependent and independent variables.
y = life_exp["LifeExpectancy"].values
X = life_exp.drop("LifeExpectancy", axis = 1).values

# Pick l1 ratio hyperparameter space

l1_space = np.linspace(0.01, 1, 30)

# Setup cross validation and parameter search
elastic = linear_model.ElasticNetCV(l1_ratio = l1_space,
                                    normalize = True, cv = 5)

# Create train and test sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 23)

# Fit to the training set

elastic.fit(X_train, y_train)

# Check linear regression R^2

r2 = elastic.score(X_test, y_test)

print("Tuned ElasticNet Alpha {}".format(elastic.alpha_))
print("Tuned ElasticNet l1 ratio {}".format(elastic.l1_ratio_))
print("Tuned R squared {}".format(r2))

Because our ElasticNet l1 ratio is just 1, we have essentially performed Lasso regression. We perform the same procedure with just lasso regression to check our results.

In [ ]:
# create lasso model with cross validation

lasso = linear_model.LassoCV(normalize = True, cv = 5)

# Fit to the training set

lasso.fit(X_train, y_train)

# Find regression R^2

r2 = lasso.score(X_test, y_test)

print("Tuned Lasso Alpha {}".format(lasso.alpha_))
print("Tuned R squared {}".format(r2))

We then plot the coefficients for the lasso regression for each column.

In [ ]:
%matplotlib inline

lasso_coef = lasso.coef_

predictors = life_exp.drop("LifeExpectancy", axis = 1).columns

plt.plot(range(len(predictors)), lasso_coef)
plt.xticks(range(len(predictors)), predictors.values, rotation = 60)
plt.ylabel("Coefficient")


## Discussion

From these results, it seems like fertility and infant mortality are most related to life expectancy. Environmental or economic variables seem to have only low influence on life expectancy. For infant mortality, life expectancy at birth would be greatly reduced if a person does not survive beyond 1 year after birth. Countries with high fertility would have many more people competing for resources, so this could be why there is such a decrease in life expectancy. Countries with high fertility also tend to be poorer, however, so the lasso regression could have arbitrary dropped GDP per capita when that was the real predictor of life expectancy.

We compare these results using Ridge regularization to see if Lasso may have been too aggressive in dropping possible predictors.

In [ ]:
%matplotlib inline
# Set up ridge regression

alpha_space = np.logspace(-2, 2, 100)

ridge = linear_model.RidgeCV(alphas = alpha_space, 
                             normalize = True, cv = 5)

# Fit to the training set

ridge.fit(X_train, y_train)

# Find regression R^2

r2 = lasso.score(X_test, y_test)

print("Tuned Ridge Alpha {}".format(ridge.alpha_))
print("Tuned R squared {}".format(r2))

ridge_coef = ridge.coef_

plt.plot(range(len(predictors)), ridge_coef)
plt.xticks(range(len(predictors)), predictors.values, rotation = 60)
plt.ylabel("Coefficient")

The $R^2$ and coefficients are not too different from the results of our lasso regression, suggesting that fertility and infant mortatility are indeed highly predictive of life expectancy.